In [ ]:
import gzip
import shutil

In [ ]:
input_gz_file = 'GSE185948_count_RNA.rds.gz'
output_rds_file = 'data_for_r.rds'

# Open the compressed file and extract it
with gzip.open(input_gz_file, 'rb') as f_in, open(output_rds_file, 'wb') as f_out:
    shutil.copyfileobj(f_in, f_out)

print(f'{input_gz_file} has been successfully uncompressed to {output_rds_file}.')

R code below

In [11]:
pip install memory_profiler

Note: you may need to restart the kernel to use updated packages.


In [61]:
import pandas as pd
from pyarrow.parquet import read_table
from scipy.sparse import coo_matrix
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, normalize
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.base import BaseEstimator, TransformerMixin
from memory_profiler import memory_usage
import matplotlib.pyplot as plt
import seaborn as sns

from kneed import KneeLocator
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import adjusted_rand_score
from sklearn.model_selection import GridSearchCV

from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.model_selection import GridSearchCV
from scipy import stats
from sklearn.cluster import HDBSCAN
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import silhouette_score
from scipy import stats


In [7]:


def load_data(data_path, row_info_path, column_info_path):
    # Load non_zero parquet data
    table = read_table(data_path)
    nonzero_data = table.to_pandas()
    
    # Adjust column indices to be 0-based
    nonzero_data['col_indices'] = nonzero_data['col_indices'] - 1
    
    # Load row and column index info
    rows = pd.read_csv(row_info_path)
    row_names = rows.iloc[:, 1].to_list()
    
    columns = pd.read_csv(column_info_path)
    column_names = columns.iloc[:, 1].to_list()
    
    # Convert the sparse matrix to a dense DataFrame
    sparse_matrix = coo_matrix(
        (nonzero_data['nonzero_elements'], (nonzero_data['row_indices'], nonzero_data['col_indices'])),
        shape=(len(row_names), len(column_names))
    )
    
    
   

    print('Returning sparse_matrix, column_names, and row_names')
    
    return sparse_matrix, column_names, row_names

In [8]:
class NormalizerSparseMatrix(TransformerMixin, BaseEstimator):
    def fit(self, X, y=None):
        # This method should typically be used to perform any necessary setup, but it's not needed for this transformer.
        return self

    def transform(self, X):
        # You can use the normalize function here
        sparse_matrix_norm = normalize(X, norm='l1', axis=1)
        return sparse_matrix_norm

In [25]:
data_path = 'non_zero.parquet'
row_info_path = 'row_names.csv'
column_info_path = 'col_names.csv'
#sparse_matrix = coo_matrix((data['nonzero_elements'], (data['row_indices'], data['col_indices'])))
sparse_matrix, row_names, column_names = load_data(data_path, row_info_path, column_info_path)

Returning sparse_matrix, column_names, and row_names


In [26]:
row_indices = np.arange(sparse_matrix.shape[0])

In [27]:
from sklearn.model_selection import train_test_split
sparse_train, sparse_test, train_indices, test_indices = train_test_split(
    sparse_matrix, row_indices, test_size=0.2, random_state=42
)
del sparse_matrix
del row_indices

In [28]:
train_row_names = [row_names[i] for i in train_indices]
test_row_names = [row_names[i] for i in test_indices]


In [31]:
pipeline = Pipeline([
        ('scaler', StandardScaler(with_mean=False)),
        ('normalizer', NormalizerSparseMatrix()),
        ('pca', TruncatedSVD(n_components=2))
    ])

In [32]:
pipeline.fit(sparse_train)

# Transform the training data
pca_sparse_train = pipeline.transform(sparse_train)

In [33]:
pca_sparse_test = pipeline.transform(sparse_test)

In [49]:
train_pca_df = pd.DataFrame(pca_sparse_train, columns=["PC1", "PC2"], index=train_row_names)
test_pca_df = pd.DataFrame(pca_sparse_test, columns=["PC1", "PC2"], index=test_row_names)

In [50]:
train_pca_df

,PC1,PC2
PKD_CATCGCTCACTCAGAT-1_3,-0.000021,0.000028
PKD_GGACGTCGTATGGGAC-1_2,-0.000012,0.000142
PKD_AGTACTGCAATGCAGG-1_3,-0.000013,0.000023
PKD_CTGCCATTCTTCGTAT-1_2,-0.000013,0.000063
PKD_CAGCAATAGTCGGCCT-1_3,-0.000016,0.000614
...,...,...
PKD_ATCCTATGTTCCTAGA-1_3,0.000235,0.000037
PKD_GGCTGTGAGGAACGAA-1_1,-0.000005,0.000027
PKD_AGGCCACCACAACCGC-1_1,-0.000011,0.000147
PKD_GGAATGGAGCCAAGGT-1_2,-0.000012,0.000043


In [51]:
test_pca_df

,PC1,PC2
PKD_CTCATCGGTTACACAC-1_2,-0.000013,0.000027
PKD_GTATTGGTCCCAATAG-1_3,0.000133,0.001408
PKD_TCCTGCAAGGACGCTA-1_2,0.000083,0.000034
PKD_GACTGATAGCACACCC-1_2,-0.000009,0.000061
PKD_GGGTCACAGTCATCGT-1_2,-0.000021,0.000022
...,...,...
PKD_GGGAAGTTCAAGCTTG-1_3,-0.000015,0.000021
PKD_CATCGGGGTAGGGAGG-1_3,-0.000018,0.000021
PKD_TCCCAGTTCAGTCCGG-1_1,-0.000011,0.000032
PKD_TGTAGACCACCGGCTA-1_1,0.000004,0.000029


In [52]:
train_pca_df.to_csv('train_pca_df.csv')
test_pca_df.to_csv('test_pca_df.csv')


## Clustering on the Train Data

In [60]:
def optimize_and_compare_kmeans(data, kmeans_params, alpha=0.05):
    # Perform Grid Search
    grid = GridSearchCV(KMeans(), kmeans_params, cv=3, refit=True)
    grid.fit(data)
    grid_search_estimator = grid.best_estimator_

    # Calculate silhouette scores
    default_kmeans = KMeans(n_clusters=2, random_state=0, n_init="auto").fit(data)
    default_silhouette_score = silhouette_score(data, default_kmeans.labels_)
    grid_search_silhouette_score = silhouette_score(data, grid_search_estimator.labels_)

    # Perform a two-sample t-test
    t_stat, p_value = stats.ttest_ind(default_kmeans.labels_, grid_search_estimator.labels_)

    # Set the default choice to "Grid Search Estimator"
    choice = "Grid Search Estimator"

    # Check if the p-value is less than the significance level
    if p_value < alpha:
        choice = "Grid Search Estimator"
    else:
        choice = "Default Parameter"

    # Output informative print statements
    print("Default KMeans Silhouette Score:", default_silhouette_score)
    print("Grid Search Estimator Silhouette Score:", grid_search_silhouette_score)

    if p_value < alpha:
        print("The difference between the two groups is statistically significant.")
        print(f"Using {choice} as it performs significantly better using a threshold of alpha = .05 .")
    else:
        print("The difference between the two groups is not statistically significant.")
        print(f"Using {choice} as there is no significant improvement using a threshold of alpha = .05.")

    return choice

# Usage example with parameters
kmeans_params = {
    'n_clusters': list(range(1, 10)),
    'init': ['random', 'k-means++'],
    'n_init': [1, 5, 10],
    'max_iter': [300],
    'random_state': [0]
}
result = optimize_and_compare_kmeans(sparse_train_pca_df, kmeans_params)

Default KMeans Silhouette Score: 0.9994378015167469
Grid Search Estimator Silhouette Score: 0.9614417750317619
The difference between the two groups is statistically significant.
Using Grid Search Estimator as it performs significantly better using a threshold of alpha = .05 .


In [ ]:
from sklearn.cluster import HDBSCAN

In [65]:
def silhouette_scorer(estimator, X):
    labels = estimator.fit_predict(X)
    if len(set(labels)) == 1:
        return 0  # Silhouette score is undefined for a single cluster
    return silhouette_score(X, labels)

def optimize_and_compare_hdbscan(data, hdbscan_params, alpha=0.05):
    # Perform Grid Search
    grid_search = GridSearchCV(
        estimator=HDBSCAN(min_cluster_size=20),
        param_grid=hdbscan_params,
        scoring=silhouette_scorer,
        cv=5,
        n_jobs=-1,
    )
    grid_search.fit(data)
    grid_search_estimator = grid_search.best_estimator_

    # Calculate silhouette scores for the default and grid search estimators
    default_hdbscan = HDBSCAN(min_cluster_size=20).fit(data)
    default_labels = default_hdbscan.labels_
    default_silhouette_score = silhouette_score(data, default_labels)

    grid_search_labels = grid_search_estimator.fit_predict(data)
    grid_search_silhouette_score = silhouette_score(data, grid_search_labels)

    # Perform a two-sample t-test
    t_stat, p_value = stats.ttest_ind(default_labels, grid_search_labels)

    # Set the default choice to "Grid Search Estimator"
    choice = "Grid Search Estimator"

    # Check if the p-value is less than the significance level
    if p_value < alpha:
        choice = "Grid Search Estimator"
    else:
        choice = "Default Parameter"

    # Output informative print statements
    print("Default HDBSCAN Silhouette Score:", default_silhouette_score)
    print("Grid Search Estimator Silhouette Score:", grid_search_silhouette_score)

    if p_value < alpha:
        print("The difference between the two groups is statistically significant.")
        print(f"Using {choice} as it performs significantly better.")
    else:
        print("The difference between the two groups is not statistically significant.")
        print(f"Using {choice} as there is no significant improvement.")

    return choice

# Define the parameter grid for HDBSCAN
hdbscan_params = {
    'min_samples': [10, 30, 50, 60, 100],
    'min_cluster_size': [100, 200, 300, 400, 500, 600],
    'cluster_selection_method': ['eom', 'leaf'],
    'metric': ['euclidean', 'manhattan']
}

# Usage example with parameters
result = optimize_and_compare_hdbscan(sparse_train_pca_df, hdbscan_params)

Default HDBSCAN Silhouette Score: 0.35331779854940065
Grid Search Estimator Silhouette Score: -0.7423886873259252
The difference between the two groups is statistically significant.
Using Grid Search Estimator as it performs significantly better.


https://towardsdatascience.com/tuning-with-hdbscan-149865ac2970

Things to consider


1) Get the clustering to work in a pipeline
2) Are these ok default parameters
3) What to do with the test data

